In [3]:
import pandas as pd

In [14]:
df = pd.read_parquet("/opt/more-new-workspace/BEBEZE03_scada_high_frequency.parquet").reset_index()[10000000:15000000]

In [15]:
df.to_csv("/opt/more-new-workspace/BEBEZE03_scada_high_frequency.csv", index=False)

In [38]:
df['sensor'] = 'sensor'
df.to_parquet("/opt/more-new-workspace/sensor2-vis.parquet", index=False, engine='pyarrow')

In [11]:
# melted_df = pd.melt(df, id_vars=['timestamp'], var_name='sensor', value_name='value')
sensor_mapping = {col: i for i, col in enumerate(df.columns[1:])}

# Melt the dataframe to reshape it and convert sensor names to integers
melted_df = pd.melt(df, id_vars=['datetime'], var_name='sensor', value_name='value')
# melted_df['sensor'] = melted_df['sensor'].map(sensor_mapping)

# Convert 'datetime' column to timestamp
melted_df['datetime'] =  pd.to_datetime(melted_df['datetime'], format='%Y-%m-%d %H:%M:%S').astype('datetime64[us]')

# Define columns to be cast to int32 and float32
int32_columns = ['sensor']  # Update 'sensor' column to int32 if needed
float32_columns = ['value']

ValueError: value_name (value) cannot match an element in the DataFrame columns.

In [20]:
melted_df.to_parquet("/opt/more-new-workspace/bebeze2-vis.parquet", index=False, engine='pyarrow')

In [27]:
df = pd.read_parquet("/opt/more-new-workspace/sensor1-vis.parquet")
df

,datetime,value,sensor
0,2023-10-17 00:00:00.000,205.3690,sensor
1,2023-10-17 00:00:00.033,205.3626,sensor
2,2023-10-17 00:00:00.067,205.3821,sensor
3,2023-10-17 00:00:00.100,205.3791,sensor
4,2023-10-17 00:00:00.133,205.3770,sensor
...,...,...,...
5395,2023-10-17 00:02:59.832,205.2544,sensor
5396,2023-10-17 00:02:59.866,205.2592,sensor
5397,2023-10-17 00:02:59.899,205.2516,sensor
5398,2023-10-17 00:02:59.932,205.2438,sensor


In [26]:
column_data_types = df.dtypes
print(column_data_types)

datetime    datetime64[us]
value              float64
sensor              object
dtype: object


In [7]:
df[df['sensor']=='irradiance']

,datetime,sensor,value
0,2012-12-20 10:40:02,irradiance,22.8
1,2012-12-20 10:45:02,irradiance,31.0
2,2012-12-20 10:50:02,irradiance,34.0
3,2012-12-20 10:55:02,irradiance,29.2
4,2012-12-20 11:00:02,irradiance,32.9
...,...,...,...
41665,2014-01-20 16:20:02,irradiance,30.5
41666,2014-01-20 16:25:02,irradiance,36.4
41667,2014-01-20 16:30:02,irradiance,36.1
41668,2014-01-20 16:35:02,irradiance,28.3


In [8]:
df.to_parquet("/opt/more-new-workspace/eugene-vis.parquet", index=False, engine='pyarrow')

In [2]:
df = pd.read_parquet("/opt/more-new-workspace/eugene-vis.parquet")
df

,datetime,sensor,value
0,2012-12-20 10:40:02,irradiance,22.8
1,2012-12-20 10:45:02,irradiance,31.0
2,2012-12-20 10:50:02,irradiance,34.0
3,2012-12-20 10:55:02,irradiance,29.2
4,2012-12-20 11:00:02,irradiance,32.9
...,...,...,...
541705,2014-01-20 16:20:02,soiling_derate,1.0
541706,2014-01-20 16:25:02,soiling_derate,1.0
541707,2014-01-20 16:30:02,soiling_derate,1.0
541708,2014-01-20 16:35:02,soiling_derate,1.0


In [21]:

flight_client = flight.FlightClient("grpc://83.212.75.52:31000")

sql = "CREATE MODEL TABLE EUG5(datetime TIMESTAMP, sensor TAG, value FIELD (0.5))"
# sql = "DROP TABLE EUG2"
action = flight.Action("CommandStatementUpdate", str.encode(sql))
result = flight_client.do_action(action)
print(list(result))

[]


In [22]:
import pyarrow
from pyarrow import flight

# Read the data into a PyArrow table.
datetime = pyarrow.array(df['datetime'])
sensor = pyarrow.array(df['sensor'])
value = pyarrow.array(df['value'])

names = ["datetime", "sensor", "value"]
table = pyarrow.Table.from_arrays([datetime, sensor, value], names=names)

# Push the table to modelardbd's Apache Arrow Flight do_put() endpoint.
upload_descriptor = flight.FlightDescriptor.for_path("EUG5")
writer, _ = flight_client.do_put(upload_descriptor, table.schema)
writer.write(table)
writer.close()

In [25]:
writer.stats

WriteStats(num_messages=2, num_record_batches=1, num_dictionary_batches=0, num_dictionary_deltas=0, num_replaced_dictionaries=0)